# Homework CAPITA SELECTA PU Learning

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import f1_score, mean_squared_error as mse
from sklearn import svm
from sklearn.linear_model import LogisticRegression

def ignore_warnings():
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=Warning)

ignore_warnings()

#from PU_Learning import *
from PU_Learning import *


## First Dataset

#### https://archive.ics.uci.edu/ml/datasets/banknote+authentication

In [2]:
data = pd.read_csv('Dataset1_train.csv')
X1_train= data.iloc[:, :-2].values
y1_train = data.iloc[:, -1].values
s1 = data.iloc[:, -2].values
c1 = Counter(s1)[1]/Counter(y1_train)[1]

data = pd.read_csv('Dataset1_test.csv')
X1_test= data.iloc[:, :-1].values
y1_test = data.iloc[:, -1].values

In [3]:
Counter(y1_test)

Counter({1: 180, 0: 232})

### Golden Standard Classifier (first dataset)

In [4]:
# Consider the dataset as fully labeled. Use this as the best case in the comparison.

golden_clf = svm.SVC(kernel='rbf',probability=True,random_state = 331).fit(np.copy(X1_train),np.copy(y1_train))
name = "Golden Standard Classifier:"

y_best_pred = golden_clf.predict(np.copy(X1_test))
y_best_prob = golden_clf.predict_proba(np.copy(X1_test))[:,1]

print(name,"F1 score:", f1_score(y1_test, y_best_pred))

Golden Standard Classifier: F1 score: 0.9917355371900827


### Non-Traditional Classifier (first dataset)

In [5]:
#fit a model on (X, s1) and see the performance. Compare it with the two methods.
non_trad_clf = svm.SVC(kernel='rbf', probability=True, random_state = 331).fit(np.copy(X1_train),np.copy(s1))
name = "Non-Traditional Classifier"

y_pred = non_trad_clf.predict(np.copy(X1_test))
y_pred_prob = non_trad_clf.predict_proba(np.copy(X1_test))[:,1]
print(name,"F1 score:", f1_score(y1_test, y_pred))
print(name,"MSE score:", mse(y_best_prob, y_pred_prob))

Non-Traditional Classifier F1 score: 0.0
Non-Traditional Classifier MSE score: 0.35758938350199343


### Spy Expectation Maximization S-EM (first dataset)

In [119]:
p = np.copy(X1_train[np.where(s1==1)])
m = np.copy(X1_train[np.where(s1==0)])


len_p = len(p)
print("Length of P:", len_p)
print("Length of M:", len(m))
index_population = np.arange(len_p)
print(index_population)

percent_spies = 0.1
Spy_size = int(np.around(len_p * percent_spies))
print("Length of Spies:", Spy_size)
Spy_index = np.random.choice(index_population, size=Spy_size, replace=False)
Spies = p[Spy_index,:]
print("Length of Spies:", len(Spies))

ms = np.concatenate((m,Spies))
index_p = np.delete(index_population,Spy_index)
print("Length of rest:", len(index_p))
p = np.array([p[i] for i in index_p])

print("Length of P:", p.shape)
print("Length of MS:", ms.shape)

Length of P: 86
Length of M: 874
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85]
Length of Spies: 9
Length of Spies: 9
Length of rest: 77
Length of P: (77, 4)
Length of MS: (883, 4)


In [123]:
from sklearn.naive_bayes import GaussianNB
# Initialize our classifier
gnb = GaussianNB()

print(ms.shape)
print(p.shape)

label_ms = np.zeros(len(ms))
label_p = np.ones(len(p))

train = np.concatenate((ms,p))
train_labels = np.concatenate((label_ms,label_p))

# Train our classifier
model = gnb.fit(train, train_labels)

old = model.score(train, train_labels)
new = old
score_variation = 1
tol = 1.0e-10
max_iter = 100
n_iter = 0

while score_variation >= tol and n_iter < max_iter:
    
    preds = gnb.predict(ms)
    train_labels = np.concatenate((preds,label_p))
    
    model = gnb.fit(train, train_labels)
    new = model.score(train, train_labels)
    
    score_variation = np.absolute(new-old)
    old = new
    n_iter += 1
    
print(new)
print("number of iterations:", n_iter)

(883, 4)
(77, 4)
0.9375
number of iterations: 7


### Above are test code

In [67]:
pu_classifier = SEM(tol = 1.0e-10, max_iter = 100, spy_prop = 0.1, l = 0.15,
                    classifier = LogisticRegression(), seed=331)
name = "SEM"

pu_classifier.fit(np.copy(X1_train), np.copy(s1))

y_pred = pu_classifier.predict(np.copy(X1_test))

y_pred_prob = pu_classifier.predict_proba(np.copy(X1_test))[:,1]

print(name,"F1 score:", f1_score(y1_test, y_pred))
print(name,"MSE score:", mse(y_best_prob, y_pred_prob))

TypeError: 'NoneType' object is not subscriptable

### Modified Logistic Regression MLR (first dataset)

In [ ]:
pu_classifier = ModifiedLogisticRegression(max_iter = 100, l_rate = 0.001, seed = 331)
name = "MLR"

pu_classifier.fit(np.copy(X1_train), np.copy(s1))

pu_classifier.estimate_c()

y_pred = pu_classifier.predict(np.copy(X1_test))

y_pred_prob = pu_classifier.predict_proba(np.copy(X1_test))

print(name,"F1 score:", f1_score(y1_test, y_pred))
print(name,"MSE score:", mse(y_best_prob, y_pred_prob))

# Second dataset

### https://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set

In [ ]:
data = pd.read_csv('Dataset2_train.csv')
X2_train= data.iloc[:, :-2].values
y2_train = data.iloc[:, -1].values
s2 = data.iloc[:, -2].values
c2 = Counter(s2)[1]/Counter(y2_train)[1]

data = pd.read_csv('Dataset2_test.csv')
X2_test= data.iloc[:, :-1].values
y2_test = data.iloc[:, -1].values

### Golden Standard Classifier (second dataset)

In [ ]:
# Consider the dataset as fully labeled. Use this as the best case in the comparison.

golden_clf = svm.SVC(kernel='rbf',probability=True,random_state = 331).fit(np.copy(X2_train),np.copy(y2_train))
name = "Golden Standard Classifier:"

y_best_pred = golden_clf.predict(np.copy(X2_test))
y_best_prob = golden_clf.predict_proba(np.copy(X2_test))[:,1]

print(name,"F1 score:", f1_score(y2_test, y_best_pred))

### Non-Traditional Classifier (second dataset)

In [ ]:
#fit a model on (X, s1) and see the performance. Compare it with the two methods.
non_trad_clf = svm.SVC(kernel='rbf', probability=True, random_state = 331).fit(np.copy(X2_train),np.copy(s2))
name = "Non-Traditional Classifier"

y_pred = non_trad_clf.predict(np.copy(X2_test))
y_pred_prob = non_trad_clf.predict_proba(np.copy(X2_test))[:,1]
print(name,"F1 score:", f1_score(y2_test, y_pred))
print(name,"MSE score:", mse(y_best_prob, y_pred_prob))

### Spy Expectation Maximization S-EM (second dataset)

In [ ]:
pu_classifier = SEM(tol = 1.0e-10, max_iter = 100, spy_prop = 0.1, l = 0.15, 
                    classifier = LogisticRegression(), seed=331)
name = "SEM"

pu_classifier.fit(np.copy(X2_train), np.copy(s2))

y_pred = pu_classifier.predict(np.copy(X2_test))

y_pred_prob = pu_classifier.predict_proba(np.copy(X2_test))[:,1]

print(name,"F1 score:", f1_score(y2_test, y_pred))
print(name,"MSE score:", mse(y_best_prob, y_pred_prob))


### Modified Logistic Regression MLR (second dataset)

In [ ]:
pu_classifier = ModifiedLogisticRegression(max_iter = 100, l_rate = 0.001, seed = 331)
name = "MLR"

pu_classifier.fit(np.copy(X2_train), np.copy(s2))

pu_classifier.estimate_c()

y_pred = pu_classifier.predict(np.copy(X2_test))

y_pred_prob = pu_classifier.predict_proba(np.copy(X2_test))

print(name,"F1 score:", f1_score(y2_test, y_pred))
print(name,"MSE score:", mse(y_best_prob, y_pred_prob))